
# <span style="color:rgb(213,80,0)"> Dofbot Control Node \- ROS2</span>

Documentacion:Torbellin Sanchez Mauricio Dariel


Fecha: 20\-10\-2025

### Descripción:

Este código implementa un nodo de ROS 2 para controlar un robot Dofbot, que es un brazo robótico de 5 grados de libertad con una pinza. El nodo ejecuta una secuencia predefinida de movimientos que incluye abrir/cerrar la pinza y mover el brazo a diferentes posiciones usando cinemática inversa.

### Dependencias:

&nbsp;&nbsp;&nbsp;&nbsp; \- ROS2 (rclpy)


&nbsp;&nbsp;&nbsp;&nbsp; \- trajectory\_msgs


&nbsp;&nbsp;&nbsp;&nbsp; \- builtin\_interfaces


| **Módulo** <br>  | **Elementos Importados** <br>  | **Descripción** <br>   |
| :-- | :-- | :-- |
| rclpy <br>  | rclpy, Node <br>  | Módulo principal de ROS 2 para Python y la clase base para crear nodos. <br>   |
| trajectory\_msgs.msg <br>  | JointTrajectory, JointTrajectoryPoint <br>  | Mensajes estándar de ROS 2 para enviar trayectorias de articulaciones. <br>   |
| builtin\_interfaces.msg <br>  | Duration <br>  | Mensaje estándar para especificar la duración (tiempo) en ROS 2. <br>   |
| time <br>  | time <br>  | Módulo estándar de Python para funciones relacionadas con el tiempo (usado con time.sleep). <br>   |
| math <br>  | cos, sin, acos, asin, atan2, sqrt <br>  | Funciones matemáticas necesarias para el cálculo de la cinemática inversa (dofbot\_ink). <br>   |



In [1]:
import rclpy
from rclpy.node import Node
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from builtin_interfaces.msg import Duration
import time
from math import cos, sin, acos, asin, atan2, sqrt

### Nodo principal de control del Dofbot.

Publica comandos de trayectoria (JointTrajectory) para mover las


articulaciones del brazo y del gripper, siguiendo una secuencia


de posiciones definidas en la función `timer_callback`.


In [2]:
class DofbotControlNode(Node):

### Constructor

In [3]:
def __init__(self):
    super().__init__("dofbot_tray_control_node")
    self.lamda_ = 0
    topic_dofbot_ = "/dofbot_trajectory_controller/joint_trajectory"
    topic_gripper_ = "/dofbot_gripper_controller/joint_trajectory"


-  `lamda_`: Contador de estado para controlar la secuencia de movimientos
-  `topic_dofbot_`: Topic para publicar trayectorias del brazo
-  `topic_gripper_`: Topic para publicar trayectorias de la pinza
### Configuración de Publishers

In [4]:
    self.dofbot_publisher_ = self.create_publisher(
    JointTrajectory, topic_dofbot_, 10)
self.dofbot_joints_ = ['arm_joint_01', 'arm_joint_02',
                        'arm_joint_03', 'arm_joint_04', 'arm_joint_05']

self.gripper_publisher_ = self.create_publisher(
    JointTrajectory, topic_gripper_, 10)
self.gripper_joints_ = ['grip_joint', 'rfinger_joint_01',
                         'rfinger_joint_02', 'lfinger_grip_joint_01',
                         'lfinger_grip_joint_02', 'lfinger_grip_joint_03']

### Estructura de Joints:
-  Brazo: 5 joints (arm\_joint\_01 a arm\_joint\_05)
-  Pinza: 6 joints que controlan los dedos y el mecanismo de agarre
### Timer y Inicialización

self.timer\_ = self.create\_timer(0.5, self.timer\_callback)


self.get\_logger().info('Nodo de control del dofbot en funcionamiento')

###  **Función** **`timer_callback`** **\- Núcleo de la Secuencia**

Esta función implementa una máquina de estados usando `lamda_` como contador:Estado 0 (λ=0): Abrir Pinza


In [5]:
gstate = 1.57  # 90° en radianes - posición abierta
gripper_st = gripper_state(gstate)


-  Configura la pinza completamente abierta
-  Espera 5 segundos
### Estado 1 (λ=1): Cerrar Pinza

In [6]:
gstate_2 = 0  # 0° - posición cerrada


-  Cierra completamente la pinza
-  Espera 5 segundos
### Estado 2 (λ=2): Abrir Pinza Nuevamente
-  Repite la apertura de la pinza
### Estado 3 (λ=3): Primera Posición del Brazo

In [7]:
x_1 = 0.2, y_1 = 0.0, z_1 = 0.05
theta_p_1 = 3.1416*(3/4)  # 135°
theta_g_1 = 0.0


-  Posición: (0.2, 0.0, 0.05) metros
-  Orientación: 135° en el plano pitch
-  Tiempo de movimiento: 2 segundos
-  Espera: 15 segundos
#### Estado 4 (λ=4): Cerrar Pinza para Agarrar
-  Cierra la pinza para agarrar un objeto
-  Tiempo: 2 segundos, Espera: 10 segundos
### Estados 5\-7 (λ=5\-7): Movimientos de Transporte

In [8]:
# Estado 5: Elevar objeto
x_2 = 0.15, y_2 = 0.0, z_2 = 0.11

# Estado 6: Mover lateralmente
x_3 = 0.15, y_3 = 0.15, z_3 = 0.11

# Estado 7: Bajar objeto
x_3 = 0.15, y_3 = 0.15, z_3 = 0.05


-  Secuencia de movimientos para transportar un objeto
-  Cada movimiento toma 2 segundos + 15 segundos de espera
### Estado 8 (λ=8): Liberar Objeto
-  Abre la pinza para soltar el objeto
-  Tiempo: 2 segundos, Espera: 10 segundos
### Estado 9 (λ=9): Posición de Reposo

In [9]:
solution_pos = [0.0, 0.0, 0.0, 0.0, 0.0]


-  Regresa todas las articulaciones a posición cero
-  Tiempo: 2 segundos, Espera: 10 segundos
### **4. Función** **`dofbot_ink`** **\- Cinemática Inversa**

Parámetros DH del Robot:

-  `z_0_1 = 0.105`: Altura base\-primera articulación
-  `L_1 = 0.084`: Longitud eslabón 1
-  `L_2 = 0.084`: Longitud eslabón 2
-  `L_3 = 0.115`: Longitud eslabón 3
### Algoritmo de Cinemática Inversa:

In [10]:
theta_1 = atan2(y_P, x_P)  # Articulación base (rotación)
aux_x = sqrt(pow(x_P, 2) + pow(y_P, 2)) - L_3*sin(theta_1_P)
aux_z = z_P - z_0_1 - L_3*cos(theta_1_P)
norm_4_P = sqrt(pow(aux_z, 2)+pow(aux_x, 2))
epsilon = acos(aux_z/norm_4_P)
alpha = acos((pow(L_1, 2)+pow(norm_4_P, 2)-pow(L_2, 2))/(2*L_1*norm_4_P))
theta_2 = epsilon - alpha
theta_3 = 3.1416 - asin((sin(alpha)*sqrt(pow(aux_x, 2) + pow(aux_z, 2)))/(L_2))
theta_4 = theta_1_P - theta_2 - theta_3
theta_5 = theta_g  # Articulación de la pinza


Retorno: `[theta_1, -theta_2, -theta_3, -theta_4, theta_5]`


 **5. Función** **`gripper_state`** **\- Control de Pinza**


In [11]:
def gripper_state(theta):
    return [float(-theta), float(theta), float(-theta),
            float(theta), float(-theta), float(theta)]


Mapeo de Articulaciones:

-  Patrón alternante de signos para coordinar movimiento simétrico de los dedos
-  Todos los joints reciben el mismo ángulo pero con orientaciones opuestas

In [12]:
def main(args=None):
    rclpy.init(args=args)
    node = DofbotControlNode()
    rclpy.spin(node)
    rclpy.shutdown()



Flujo de Ejecución:

1.  Inicializa ROS 2
2. Crea instancia del nodo
3. Ejecuta el bucle principal (spin)
4. Limpieza al finalizar

**Topics Publicados:**

-  `/dofbot_trajectory_controller/joint_trajectory`: Control del brazo
-  `/dofbot_gripper_controller/joint_trajectory`: Control de la pinza

**Mensajes Utilizados:**

-  `JointTrajectory`: Define trayectorias para múltiples joints
-  `JointTrajectoryPoint`: Puntos específicos en la trayectoria
-  `Duration`: Especifica tiempos de ejecución

**Características de la Secuencia:**

-  Duración total: ~105 segundos (sin contar delays internos)
-  Movimientos del brazo: 5 posiciones diferentes
-  Estados de pinza: 4 cambios (abrir/cerrar)
-  Tolerancia de tiempo: Usa `time.sleep()` para sincronización
